In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split



In [69]:
#global variables


SEED = 42

FOLDS = 5

FILEPATH = '../data/'

In [70]:
train_df =pd.read_csv(f"{FILEPATH}train.csv")
test_df = pd.read_csv(f"{FILEPATH}test.csv")
original_df = pd.read_csv(f"{FILEPATH}ObesityDataSet.csv")

test_ids = test_df.copy()
test_df = test_df.drop(["id"], axis = 1)
train_df = train_df.drop(["id"], axis = 1)


# display(train_df.head())
# display(original_df.head())
# display(id.head())
# display(train_df.shape)
# display(original_df.shape)
# # Get the unique columns from each data frame
# train_columns = set(train_df.columns)
# original_columns = set(original_df.columns)

# # Find the difference in columns
# difference_columns = train_columns.symmetric_difference(original_columns)

# # Display the difference in columns
# print("Columns present in train_df but not in original_df:")
# print(difference_columns.intersection(train_columns))
# print("\nColumns present in original_df but not in train_df:")
# print(difference_columns.intersection(original_columns))


# train_df = pd.concat([train_df, original_df], ignore_index=True)
# display(train_df.head())
# display(train_df.shape)

In [ ]:
def analyze_dataframe(df):
    """
    Analyze a pandas DataFrame and provide a summary of its characteristics.

    Parameters:
    df (pandas.DataFrame): The input DataFrame to analyze.

    Returns:
    None
    """
    print("DataFrame Information:")
    print("______________________")
    display(df.info(verbose=True, show_counts=True))
    print("\n")
    
    print("DataFrame Head:")
    print("______________________")
    display(df.head())
    print("\n")

    print("DataFrame Tail:")
    print("______________________")
    display(df.tail())
    print("\n")

    print("DataFrame Description:")
    print("______________________")
    display(df.describe().T)
    print("\n")

    print("Number of Null Values:")
    print("______________________")
    display(df.isnull().sum())
    print("\n")

    print("Number of Duplicated Rows:")
    print("______________________")
    display(df.duplicated().sum())
    print("\n")

    print("Number of Unique Values:")
    print("______________________")
    display(df.nunique())
    print("\n")

    print("DataFrame Shape:")
    print("______________________")
    print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
    print("\n")
    
    print("DataFrame Columns:")
    print("______________________")
    display(df.columns)
    

analyze_dataframe(train)

# Feature Engineering
___

In [71]:

def preprocess_data(df, cat_features, num_features, scaler):
    
    # One-hot encode categorical features
    df = pd.get_dummies(df, columns=cat_features)

    # Normalize numerical features
    df[num_features] = scaler.fit_transform(df[num_features])


    return df

In [72]:
cat_features = [
    "Gender",
    "family_history_with_overweight",
    "FAVC",
    "CAEC",
    "SMOKE",
    "SCC",
    "CALC",
    "MTRANS",
]
num_features = [
    "Age",
    "Height",
    "Weight",
    "FCVC",
    "NCP",
    "CH2O",
    "FAF",
    "TUE",
]


In [73]:
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score,confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold


scaler = StandardScaler()

# Preprocess the training data
train_df = preprocess_data(train_df, cat_features, num_features, scaler)
test_df = preprocess_data(test_df, cat_features, num_features, scaler)

# Initialize the label encoder
label_encoder = LabelEncoder()
X_train = train_df.drop('NObeyesdad', axis=1)

# Fit and transform the labels to integers
y_train_encoded = label_encoder.fit_transform(train_df['NObeyesdad'])

# Initialize the stratified k-fold cross-validator
skf = StratifiedKFold(n_splits=FOLDS, random_state=SEED, shuffle=True)

# Split the data into training and validation sets using stratified k-fold cross-validation
for train_index, val_index in skf.split(X_train, y_train_encoded):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train_encoded[train_index], y_train_encoded[val_index]

    # Train the XGBoost model
    model = xgb.XGBClassifier(n_estimators=750, random_state=SEED)

    # Fit the XGBoost model
    model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation set
    y_pred_fold = model.predict(X_val_fold)

    # Calculate accuracy for each fold
    accuracy_fold = accuracy_score(y_val_fold, y_pred_fold)
    print(f'Accuracy for fold: {accuracy_fold}')

Accuracy for fold: 0.9075144508670521
Accuracy for fold: 0.8952312138728323
Accuracy for fold: 0.9029383429672447
Accuracy for fold: 0.8935196338231751
Accuracy for fold: 0.897133220910624


In [76]:
# Assuming 'test_df' has been preprocessed in a similar manner to 'train_df'
# Make predictions on the test set to get class labels directly
test_pred_class_integers = model.predict(test_df)

# Inverse transform to get original string labels
test_pred_labels = label_encoder.inverse_transform(test_pred_class_integers)

# Create submission DataFrame
submission_df = pd.DataFrame({
    'id': test_ids['id'],
    'NObeyesdad': test_pred_labels
})

# Save the DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

# Display the first few rows of the submission DataFrame
submission_df.head()

ValueError: feature_names mismatch: ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE', 'Gender_Female', 'Gender_Male', 'family_history_with_overweight_no', 'family_history_with_overweight_yes', 'FAVC_no', 'FAVC_yes', 'CAEC_Always', 'CAEC_Frequently', 'CAEC_Sometimes', 'CAEC_no', 'SMOKE_no', 'SMOKE_yes', 'SCC_no', 'SCC_yes', 'CALC_Frequently', 'CALC_Sometimes', 'CALC_no', 'MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike', 'MTRANS_Public_Transportation', 'MTRANS_Walking'] ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE', 'Gender_Female', 'Gender_Male', 'family_history_with_overweight_no', 'family_history_with_overweight_yes', 'FAVC_no', 'FAVC_yes', 'CAEC_Always', 'CAEC_Frequently', 'CAEC_Sometimes', 'CAEC_no', 'SMOKE_no', 'SMOKE_yes', 'SCC_no', 'SCC_yes', 'CALC_Always', 'CALC_Frequently', 'CALC_Sometimes', 'CALC_no', 'MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike', 'MTRANS_Public_Transportation', 'MTRANS_Walking']
training data did not have the fo